In [1]:
from init import *

In [2]:
code = r'''
#include <eosio/crypto.hpp>
#include <eosio/print.hpp>
#include <eosio/name.hpp>

using namespace eosio;

#define N(name) name##_n.value

extern "C" {
    void apply( uint64_t receiver, uint64_t code, uint64_t action ) {
        checksum160 hash160 = sha1("hello,worldd", 12);
        printhex((void *)hash160.data(), 20);print("\n");

        assert_sha1("hello,worldd", 12, hash160);

        checksum256 hash256 = sha256("hello,world", 12);
        printhex((void *)hash256.data(), 32);print("\n");
        assert_sha256("hello,world", 12, hash256);

        checksum512 hash512 = sha512("hello,world", 12);
        printhex((void *)hash512.data(), 64);print("\n");
        assert_sha512("hello,world", 12, hash512);

        checksum160 hash = ripemd160("hello,world", 12);
        printhex((void *)hash.data(), 20);print("\n");
        assert_ripemd160("hello,world", 12, hash);
    }
}
'''

code = uuosapi.compile(test_account1, code, vm_type=0)
uuosapi.deploy_code(test_account1, code, vm_type=0)
print('done!')

done!


In [4]:
r = uuosapi.push_action(test_account1, 'sayhello', b'hello,world', {test_account1:'active'})
print(r['processed']['action_traces'][0]['console'])

dccf48347b12434e0b8568d6b7de7d4000000000
5d78ff1f609563113144c53353b5c67dca8a6203b8b1ba8c33a58c7858cef3aa
9124d391e6ea07442f378898c85a2521034ed16c17378ff6ed3fa37b0dea8b12dee106e6d87b0d5d74685d7cfbc7b0ecdcba0c9ed3126fd2e2591a174d343cb4
9fbcc36bef806c7caaec414590db060900000000



In [10]:
code = r'''
#include <vector>
#include <eosio/action.hpp>
#include <eosio/crypto.hpp>
#include <eosio/print.hpp>
#include <eosio/name.hpp>

using namespace std;
using namespace eosio;

#define N(name) name##_n.value

struct mystruct {
    vector<char> text;
    signature sign;
    EOSLIB_SERIALIZE( mystruct, (text)(sign) )
};

extern "C" {
    void apply( uint64_t receiver, uint64_t code, uint64_t action ) {
        mystruct s = unpack_action_data<mystruct>();
        checksum256 digest = sha256(s.text.data(), s.text.size());
        public_key key = recover_key(digest, s.sign);
        printhex((void *)std::get<0>(key).data(), std::get<0>(key).size());
        assert_recover_key(digest, s.sign, key);
    }
}
'''

abi = '''
{
   "version": "eosio::abi/1.0",
   "types": [],
  "structs": [{
      "name": "sayhello",
      "base": "",
      "fields": [
        {"name":"text", "type":"bytes"},
        {"name":"sign", "type":"signature"}
        ]
    }
  ],
  "actions": [{
      "name": "sayhello",
      "type": "sayhello",
      "ricardian_contract": ""
    }
  ],
  "tables": [
  ]
}
'''

code = uuosapi.compile(test_account1, code, vm_type=0)
uuosapi.deploy_code(test_account1, code, vm_type=0)
print('done!')

done!


In [15]:
uuosapi.set_abi(test_account1, abi)

In [17]:
uuosapi.get_account(test_account1)

{'account_name': 'wkpmdjdsztyu',
 'head_block_num': 45482171,
 'head_block_time': '2021-01-12T03:31:09.000',
 'privileged': False,
 'last_code_update': '2021-01-12T03:28:43.500',
 'created': '2020-11-30T02:21:01.000',
 'core_liquid_balance': '54.2551 TNT',
 'ram_quota': 1190188,
 'net_weight': 110000,
 'cpu_weight': 110000,
 'net_limit': {'used': 199847, 'available': 26158847, 'max': 26358694},
 'cpu_limit': {'used': 173871, 'available': 4853169, 'max': 5027040},
 'ram_usage': 163524,
 'permissions': [{'perm_name': 'active',
   'parent': 'owner',
   'required_auth': {'threshold': 1,
    'keys': [{'key': 'EOS4yV91cqR6db8ntKtCLFVwWfvvvDMZctpd85GsaX5GUMaiqUWYL',
      'weight': 1}],
    'accounts': [{'permission': {'actor': 'wkpmdjdsztyu',
       'permission': 'eosio.code'},
      'weight': 1}],
    'waits': []}},
  {'perm_name': 'owner',
   'parent': '',
   'required_auth': {'threshold': 1,
    'keys': [{'key': 'EOS7iPioB1Uscn14dYdw5PrqAHVU2F54Bi3kJ1qr6WbRe1tMtHGEo',
      'weight': 1}],

In [18]:
text = b'hello,world'
import hashlib
h = hashlib.sha256()
h.update(text)
sign = wallet.sign_digest(h.digest(), 'EOS4yV91cqR6db8ntKtCLFVwWfvvvDMZctpd85GsaX5GUMaiqUWYL')
assert sign
#    args = {'text':text.hex(), 'sign': sign}
args = {'text':text.hex()}
args = {'sign': sign}
args = {'text':text.hex(), 'sign': sign}
r = uuosapi.push_action(test_account1, 'sayhello', args, {test_account1:'active'})
print(r['processed']['action_traces'][0]['console'])


020b583ea8dd7626801a83a0d421ea442feeedb7c3a3cf08db9f61ea6612faa9f7


In [19]:
import base58

public_key = bytes.fromhex('020b583ea8dd7626801a83a0d421ea442feeedb7c3a3cf08db9f61ea6612faa9f7')
print(public_key)

h = hashlib.new('ripemd160')
h.update(public_key)
digest = h.digest()
public_key += digest[:4]
b'EOS'+base58.b58encode(public_key)

b'\x02\x0bX>\xa8\xddv&\x80\x1a\x83\xa0\xd4!\xeaD/\xee\xed\xb7\xc3\xa3\xcf\x08\xdb\x9fa\xeaf\x12\xfa\xa9\xf7'


b'EOS4yV91cqR6db8ntKtCLFVwWfvvvDMZctpd85GsaX5GUMaiqUWYL'